In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from ipywidgets import interact, widgets, Button, VBox
import geopandas as gpd
from scipy.interpolate import griddata
from IPython.display import display

In [6]:
files = ['gdańsk_weather_data_final.csv', 'kraków_weather_data_final.csv', 'warszawa_weather_data_final.csv', 'wrocław_weather_data_final.csv', 'szczecin_weather_data_final.csv']
city_coordinates = {
    'Gdańsk': (54.3520, 18.6466),
    'Kraków': (50.0647, 19.9450),
    'Warszawa': (52.2297, 21.0122),
    'Wrocław': (51.1079, 17.0385),
    'Szczecin': (53.4285, 14.5528)
}

In [7]:
data_frames = []
for file in files:
    df = pd.read_csv(file)
    df['time'] = pd.to_datetime(df['time'])
    data_frames.append(df)

df = pd.concat(data_frames, ignore_index=True)

# Debug: Wyświetl pierwsze kilka wierszy wczytanych danych
print(df.head())

# Wczytaj granice Polski za pomocą geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
poland = world[world.name == 'Poland']

# Globalna zmienna do przechowywania aktualnego wykresu
current_figure = None


        time      tavg      prcp  wspd    pres    City  number
0 2024-01-01  1.900000 -0.162265  10.4  1004.6  Gdańsk       1
1 2024-01-02  1.500000  2.000000   9.9  1003.2  Gdańsk       2
2 2024-01-03 -4.344857  0.360638  19.5   992.8  Gdańsk       3
3 2024-01-04 -3.800000  7.600000  15.3   993.3  Gdańsk       4
4 2024-01-05 -5.700000  0.000000  12.9  1007.5  Gdańsk       5


C:\Users\ssiko\AppData\Local\Temp\ipykernel_12876\3068955848.py:13: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [8]:
def plot_heatmap(selected_date, variable):
    global current_figure

    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([13.5, 24.5, 48, 55])  # Zmniejsz zasięg mapy, aby pokryć tylko Polskę z niewielkim marginesem
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')

    # Dodanie granic Polski
    poland_boundary = poland.geometry.unary_union
    ax.add_geometries([poland_boundary], ccrs.PlateCarree(), facecolor='none', edgecolor='black')

    # Filtruj dane według wybranej daty
    filtered_data = df[df['time'].dt.date == selected_date]

    # Debug: Wyświetl filtrowane dane
    print(f"Selected date: {selected_date}")
    print(filtered_data)

    lats = []
    lons = []
    values = []

    for idx, row in filtered_data.iterrows():
        city = row['City']
        lat, lon = city_coordinates[city]

        lats.append(lat)
        lons.append(lon)
        values.append(row[variable])

    # Przekształcenie danych na siatkę do heatmapy
    xi, yi = np.meshgrid(np.linspace(min(lons)-5, max(lons)+5, 500), np.linspace(min(lats)-5, max(lats)+5, 500))

    # Interpolacja danych do siatki
    grid = griddata((lons, lats), values, (xi, yi), method='cubic')

    # Wybór palety kolorów
    if variable == 'tavg':
        cmap = 'coolwarm'  # odcienie czerwone i niebieskie
    elif variable == 'prcp':
        cmap = 'Blues'     # odcienie niebieskiego
    elif variable == 'pres':
        cmap = 'viridis'   # inne kolory
    elif variable == 'wspd':
        cmap = 'cividis'  # odcienie tęczy, dla ciekawego efektu

    # Rysowanie heatmapy
    contour = ax.contourf(xi, yi, grid, levels=100, cmap=cmap, alpha=0.8, transform=ccrs.PlateCarree())
    fig.colorbar(contour, ax=ax, orientation='vertical', label=variable.capitalize())

    plt.title(f"{variable.capitalize()} Heatmap for {selected_date.strftime('%Y-%m-%d')}")
    plt.show()

    current_figure = fig

# Funkcja do zapisywania wykresu
def save_current_figure(_):
    if current_figure:
        current_figure.savefig('weather_heatmap.png')
        print("Wykres zapisany jako 'weather_heatmap.png'")
    else:
        print("Brak wykresu do zapisania")

# Znajdź unikalne daty
unique_dates = df['time'].dt.date.unique()

# Debug: Wyświetl unikalne daty
print(unique_dates)

# Utwórz interaktywne suwaki
date_slider = widgets.SelectionSlider(options=unique_dates, description="Date", continuous_update=False)
variable_dropdown = widgets.Dropdown(options=['tavg', 'prcp', 'pres', 'wspd'], description="Variable")

interact(plot_heatmap, selected_date=date_slider, variable=variable_dropdown)

# Utwórz przycisk do zapisywania wykresu
save_button = Button(description="Save Heatmap")
save_button.on_click(save_current_figure)

# Wyświetl suwaki i przycisk
display(VBox([date_slider, variable_dropdown, save_button]))

[datetime.date(2024, 1, 1) datetime.date(2024, 1, 2)
 datetime.date(2024, 1, 3) datetime.date(2024, 1, 4)
 datetime.date(2024, 1, 5) datetime.date(2024, 1, 6)
 datetime.date(2024, 1, 7) datetime.date(2024, 1, 8)
 datetime.date(2024, 1, 9) datetime.date(2024, 1, 10)
 datetime.date(2024, 1, 11) datetime.date(2024, 1, 12)
 datetime.date(2024, 1, 13) datetime.date(2024, 1, 14)
 datetime.date(2024, 1, 15) datetime.date(2024, 1, 16)
 datetime.date(2024, 1, 17) datetime.date(2024, 1, 18)
 datetime.date(2024, 1, 19) datetime.date(2024, 1, 20)
 datetime.date(2024, 1, 21) datetime.date(2024, 1, 22)
 datetime.date(2024, 1, 23) datetime.date(2024, 1, 24)
 datetime.date(2024, 1, 25) datetime.date(2024, 1, 26)
 datetime.date(2024, 1, 27) datetime.date(2024, 1, 28)
 datetime.date(2024, 1, 29) datetime.date(2024, 1, 30)
 datetime.date(2024, 1, 31) datetime.date(2024, 2, 1)
 datetime.date(2024, 2, 2) datetime.date(2024, 2, 3)
 datetime.date(2024, 2, 4) datetime.date(2024, 2, 5)
 datetime.date(2024, 2, 